<a href="https://colab.research.google.com/github/peterhanlon/NLP-with-Python/blob/master/nemo_asr_quartznet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

In [ ]:
!pip install librosa

In [ ]:
# NeMo's "core" package
import nemo
# NeMo's ASR collection - this collections contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr
import nemo.collections.nlp as nemo_nlp

In [55]:
# This line will download pre-trained QuartzNet15x5 model from NVIDIA's NGC cloud and instantiate it for you
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En")

[NeMo I 2021-01-02 15:54:49 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.0.0b4/QuartzNet15x5Base-En/00869f9c89b8393ca3de640e0c536bd2/QuartzNet15x5Base-En.nemo.
[NeMo I 2021-01-02 15:54:49 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.0.0b4/QuartzNet15x5Base-En/00869f9c89b8393ca3de640e0c536bd2/QuartzNet15x5Base-En.nemo
[NeMo I 2021-01-02 15:54:49 common:423] Instantiating model from pre-trained checkpoint
[NeMo I 2021-01-02 15:54:50 features:235] PADDING: 16
[NeMo I 2021-01-02 15:54:50 features:251] STFT using torch
[NeMo I 2021-01-02 15:54:51 modelPT:402] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.0b4/QuartzNet15x5Base-En/00869f9c89b8393ca3de640e0c536bd2/QuartzNet15x5Base-En.nemo.


In [57]:
# Punctuation and capitalization model
punctuation = nemo_nlp.models.PunctuationCapitalizationModel.from_pretrained(model_name='Punctuation_Capitalization_with_DistilBERT')

[NeMo I 2021-01-02 15:56:09 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.0.0b4/Punctuation_Capitalization_with_DistilBERT/6906a3705a7989aa9995aed680fb5d22/Punctuation_Capitalization_with_DistilBERT.nemo.
[NeMo I 2021-01-02 15:56:09 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.0.0b4/Punctuation_Capitalization_with_DistilBERT/6906a3705a7989aa9995aed680fb5d22/Punctuation_Capitalization_with_DistilBERT.nemo
[NeMo I 2021-01-02 15:56:09 common:423] Instantiating model from pre-trained checkpoint


Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.
[NeMo W 2021-01-02 15:56:13 modelPT:146] Please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    text_file: text_train.txt
    labels_file: labels_train.txt
    shuffle: true
    num_samples: -1
    batch_size: 64
    
[NeMo W 2021-01-02 15:56:13 modelPT:153] Please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    ds_item: null
    text_file: text_dev.txt
    labels_file: labels_dev.txt
    shuffle: false
    num_samples: -1
    batch_size: 64
    
[NeMo W 2021-01-02 15:56:13 modelPT:1175] World size can only be set by PyTorch Lightning Trainer.


[NeMo I 2021-01-02 15:56:17 modelPT:402] Model PunctuationCapitalizationModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.0b4/Punctuation_Capitalization_with_DistilBERT/6906a3705a7989aa9995aed680fb5d22/Punctuation_Capitalization_with_DistilBERT.nemo.


Use the NeMo punctuation model to puctuctuate the text

In [59]:
def punctuation(raw_text):
  res = punctuation.add_punctuation_capitalization(queries=[raw_text])
  return res[0]


[NeMo I 2021-01-02 15:56:50 punctuation_capitalization_model:374] Using batch size 1 for inference
[NeMo I 2021-01-02 15:56:50 punctuation_capitalization_dataset:134] Max length: 10
[NeMo I 2021-01-02 15:56:50 data_preprocessing:295] Some stats of the lengths of the sequences:
[NeMo I 2021-01-02 15:56:50 data_preprocessing:301] Min: 10 |                  Max: 10 |                  Mean: 10.0 |                  Median: 10.0
[NeMo I 2021-01-02 15:56:50 data_preprocessing:303] 75 percentile: 10.00
[NeMo I 2021-01-02 15:56:50 data_preprocessing:304] 99 percentile: 10.00
[NeMo I 2021-01-02 15:56:50 punctuation_capitalization_dataset:165] 0 are longer than 10
[NeMo I 2021-01-02 15:56:50 punctuation_capitalization_dataset:168] *** Example ***
[NeMo I 2021-01-02 15:56:50 punctuation_capitalization_dataset:169] i: 0
[NeMo I 2021-01-02 15:56:50 punctuation_capitalization_dataset:170] subtokens: [CLS] this is a test my name is peter [SEP]
[NeMo I 2021-01-02 15:56:50 punctuation_capitalization_dat

In [48]:
from scipy.io import wavfile
fs, data = wavfile.read('/content/drive/MyDrive/audio/example.wav')
wavfile.write('/content/drive/MyDrive/audio/example_channel_0.wav', fs, data[:, 0])

This Version used SOX 
You must have mono audio with a sample rate of 16000. If not use the following command to fix the wav file.
sox ./example.wav ./example-resample.wav rate 16000 remix 1

In [39]:
files = ['/content/drive/MyDrive/audio/example_resampled_sox.wav']
transcription = quartznet.transcribe(paths2audio_files=files)

for trans in transcription:
  print(trans)

good morning teach johnsinco how can i help youokay are you looking for a property to buy o are you looking for a proparcy to rent okay so thires one specific so do you want to organize a viewing on that prophete okay what's your name please and you sa name's faceokay and which propecy is that you interested in does it give you a road name es it does it not give you the name of the roadand how much is um fulland it still runs yeahis that three foull six seven a month okay and umd so when would you be looking for a viewing when when would you like to view itokaywhat was the name of the building adain sorry i'm just trying to locate the detailsokay so i'd need to check with i see what we can organize for a viewing four yame um afraid the office is close the day due to a being a bank holiday i can have somebody give you a call back tomorrow to see what can be arranged for youright okay um okay what's the number yep yeah yeah okay and um do you have any email addresses another points of co

In [51]:
from pydub import AudioSegment
sound = AudioSegment.from_file("/content/drive/MyDrive/audio/example_channel_0.wav", format="wav")
sound = sound.set_frame_rate(16000)

# more complex export
file_handle = sound.export("/content/drive/MyDrive/audio/example_resampled_pydub.wav",
                           format="wav",
                           bitrate="16k")


In [52]:
files = ['/content/drive/MyDrive/audio/example_resampled_pydub.wav']
transcription = quartznet.transcribe(paths2audio_files=files)

for trans in transcription:
  print(trans)

good morning teach juhmpsind cao how can i help youokay are you looking for a property to buy or are you looking for a property to rent okay so there's one specific say do you want to organize a viewing on that properta  what's your name tlease and you finane fave okay in which property is at you interested in does it give you a road name it does it not give you the name of the roadand how much is it im foraits sevenampis hat three four six seven a monthand then so when would you be looking for a vewing when when would you like to view itokay what was the name of the building again sorry i'm just trying to locate the detailokay so i'd meed to check with a tea what we can organize firviewing four yam am afraid the offices close the day due to a binga bank holiday i can have somebody give you a call back that tomorrow to see what can be arranged for youlike okay i'm okay what's the number yep yep yep okay and graham do you have en email addresses and other points of contactokay jistoneou

Extract a single channel from the wavfile

In [49]:

import librosa
y, sr = librosa.load('/content/drive/MyDrive/audio/example_channel_0.wav')
data = librosa.resample(y, sr, 16000)
librosa.output.write_wav('/content/drive/MyDrive/audio/example_resample_librosa.wav', data, 16000)


In [60]:
files = ['/content/drive/MyDrive/audio/example_resample_librosa.wav']
transcription = quartznet.transcribe(paths2audio_files=files)

for trans in transcription:
  print(punctuation(trans))

TypeError: ignored